In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
glove = {}
f = open('data/mini_glove.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove[word] = coefs
f.close()

In [3]:
class Glove2kSparse(nn.Module):
    def __init__(self):
        super(Glove2kSparse, self).__init__()
        self.encoder = nn.Linear(100, 1000)
        self.decoder = nn.Linear(1000, 100)
    
    def encode(self, x):
        x = self.encoder(x)
        topk = torch.topk(x, 25).indices
        debug_count = 0
        for index in range(len(x)):
            if index not in topk:
                x[index] = 0
                debug_count += 1
        return x
    
    def forward(self, x):  # Unsqueeze / squeeze?
        return self.decoder(self.encode(x))
    
    def view_grads(self):
        enc_grads = net.encoder.weight.grad.resize(100000)
        dec_grads = net.decoder.weight.grad.resize(100000)
        return sparse_vec_to_dict(torch.cat([enc_grads, dec_grads]))
        
net = Glove2kSparse()

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1):
    epoch_loss = 0
    for j in range(len(glove)):
        x_var = torch.tensor(list(glove.values())[j]).squeeze(0)
        optimizer.zero_grad()
        xpred_var = net(x_var)
        loss = criterion(xpred_var, x_var)
        loss.backward()
        optimizer.step()
        
        # running loss
        epoch_loss += loss.item()
        if j%1000 == 0 and j > 0:
            print(j, "   ", epoch_loss / j)
    print(f"Epoch {epoch} Average Loss:  {epoch_loss / 10000}")

In [99]:
# torch.save(net.state_dict(), "data/very_sparse_glove.pt")
net.load_state_dict(torch.load("data/very_sparse_glove.pt"))

<All keys matched successfully>

In [1]:
def word2sparse(word):
    return net.encode(torch.tensor(glove[word])).squeeze().detach().numpy()

def vec_dist(vec1, vec2):
    diff = vec1 - vec2
    return np.sqrt(np.dot(diff, diff))

def sparse_similarity(vec1, vec2):
    count = 0
    for index in range(len(vec1)):
        if vec1[index] != 0 and vec2[index] != 0:
            count += 1
    return count

def show_sparse(vec):
    output = {}
    for index in range(len(vec)):
        if vec[index] != 0:
            output[index] = vec[index]
    return output